In [0]:
import requests
from datetime import datetime
from pyspark.sql import functions as F
# Configurações
catalogo = "medalhao"
bronze_db_name = "bronze"
landing_schema_name = "default"
landing_volume_name = "landing"

# Caminho base para os arquivos CSV na Landing
landing_path = f"/Volumes/{catalogo}/{landing_schema_name}/{landing_volume_name}"

# Garantir que o catálogo e o schema bronze existam
spark.sql(f"USE CATALOG {catalogo}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {bronze_db_name}")

print(f"Contexto configurado para Catálogo: {catalogo}, Schema: {bronze_db_name}")
print(f"Lendo arquivos da Landing em: {landing_path}")
print("---")

In [0]:
# Ingestão de Arquivos
arquivos_csv = {
    "olist_customers_dataset.csv": "ft_consumidores",
    "olist_geolocation_dataset.csv": "ft_geolocalizacao",
    "olist_order_items_dataset.csv": "ft_itens_pedidos",
    "olist_order_payments_dataset.csv": "ft_pagamentos_pedidos",
    "olist_order_reviews_dataset.csv": "ft_avaliacoes_pedidos",
    "olist_orders_dataset.csv": "ft_pedidos",
    "olist_products_dataset.csv": "ft_produtos",
    "olist_sellers_dataset.csv": "ft_vendedores",
    "product_category_name_translation.csv": "dim_categoria_produtos_traducao"
}

for nome_arquivo, nome_tabela in arquivos_csv.items():
    try:
        print(f"➡️ Processando: {nome_arquivo} → {bronze_db_name}.{nome_tabela}")
        
        caminho_arquivo = f"{landing_path}/{nome_arquivo}"
        
        df = (
            spark.read
            .csv(caminho_arquivo, header=True, inferSchema=True)
            .withColumn("ingestion_timestamp", F.current_timestamp())
        )
        
        # --- CORREÇÃO ---
        # Salva diretamente como Tabela Gerenciada no schema 'bronze'
        # Usamos o nome completo (catalogo.schema.tabela) para garantir
        nome_completo_tabela = f"{catalogo}.{bronze_db_name}.{nome_tabela}"
        df.write.format("delta").mode("overwrite").saveAsTable(nome_completo_tabela)
        
        displayHTML(f"<span style='color:limegreen; font-weight:bold;'> Tabela CSV criada com sucesso:</span> {nome_completo_tabela}")
    
    except Exception as e:
        displayHTML(f"<span style='color:red; font-weight:bold;'> Erro ao criar tabela CSV:</span> {nome_completo_tabela}<br><b>Detalhes:</b> {str(e)}")

print("---")
print("Ingestão de CSVs concluída.")
print("---")

In [0]:
# Ingestão da Cotação do Dólar
data_inicio_formatada = "11-03-2025" 
data_fim_formatada = "11-07-2025"   

# Tabela destino
tabela_dolar = "dm_cotacao_dolar"
nome_completo_tabela_dolar = f"{catalogo}.{bronze_db_name}.{tabela_dolar}"

url = (
    "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
    "CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
    f"@dataInicial='{data_inicio_formatada}'&@dataFinalCotacao='{data_fim_formatada}'"
    "&$select=dataHoraCotacao,cotacaoCompra&$format=json"
)

try:
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()["value"]
        
        if not data:
             displayHTML(f"<h3 style='color:orange;'> API não retornou dados para o período.</h3>"
                         f"<p>Verifique as datas: {data_inicio_formatada} a {data_fim_formatada}</p>")
        else:
            # Criação do DataFrame
            df_dolar = spark.createDataFrame(data)
            df_dolar = df_dolar.withColumn("ingestion_timestamp", F.current_timestamp())
            
            # Salva diretamente como Tabela Gerenciada
            df_dolar.write.format("delta").mode("overwrite").saveAsTable(nome_completo_tabela_dolar)
            
            # Informações de sucesso
            qtd_registros = df_dolar.count()
            ultima_data = df_dolar.agg(F.max("dataHoraCotacao")).first()[0]
            
            displayHTML(f"""
            <h3 style='color:limegreen;'> Cotação do Dólar carregada com sucesso!</h3>
            <ul>
                <li><b>Tabela:</b> {nome_completo_tabela_dolar}</li>
                <li><b>Período:</b> {data_inicio_formatada} → {data_fim_formatada}</li>
                <li><b>Registros carregados:</b> {qtd_registros}</li>
                <li><b>Última data de cotação:</b> {ultima_data}</li>
            </ul>
            """)
    
    else:
        displayHTML(f"""
        <h3 style='color:red;'> Erro ao consultar API do Banco Central</h3>
        <p>Status HTTP: {response.status_code}</p>
        <p>URL: {url}</p>
        <p>Verifique a URL ou as datas informadas.</p>
        """)

except Exception as e:
     displayHTML(f"<span style='color:red; font-weight:bold;'> Erro ao criar tabela de Cotação:</span> {nome_completo_tabela_dolar}<br><b>Detalhes:</b> {str(e)}")

print("---")
print("Processo de ingestão para a camada Bronze concluído.")